# Ejercicios Pair Programming - Módulo 2 Sprint 2
## Lección 2 - Limpieza II
### Natalia García Lago y Noemi Azorín 

In [56]:
import pandas as pd 
import regex as re 
import numpy as np 

Antes de seguir, recordamos las preguntas que nos planteamos al principio del _pair programming_ de EDA para dirigir nuestro análisis.

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?

- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?

- Independientemente de la edad, sufren los hombres más ataques que las mujeres?

- ¿En qué mes ocurren más ataques?

- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?

- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

- De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex. Si reducimos esto a una tabla para saber que tenemos ya limpito y que no. 

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra.

Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉. Para esto trabajaremos con el fichero generado en Limpieza I.

1. Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.

    - Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

    - Tendremos que crearnos una función que aplicaremos sobre nuestra columna species para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [41]:
#Abrimos el archivo 
ataques = pd.read_csv("../ficheros/ataques_limpiezaI.csv", index_col = 0)

In [42]:
patron_blanco = ".*[Ww](hite|HITE).*"
patron_grey = ".*[Gg](rey|REY).*"
patron_lemon = ".*[Ll](emon|EMON).*"
patron_toro = ".*[Bb](ull|ULL).*"


In [43]:
def especies (col):
    try:
        if len(re.findall(patron_blanco, col)) > 0:
            return "Tiburón Blanco"
        elif len(re.findall(patron_grey, col)) > 0:
            return "Tiburón Gris"
        elif len(re.findall(patron_lemon, col)) > 0:
            return "Tiburón limón"
        elif len(re.findall(patron_toro, regex = True)) > 0:
            return "Tiburón toro"
    except:
        return "Unspecified"

In [44]:
ataques.columns

Index(['case_number', 'year', 'type', 'country', 'activity', 'age', 'species_',
       'mes', 'fatal', 'sex'],
      dtype='object')

In [46]:
ataques["especies"] = ataques["species_"].apply(especies)
ataques.head()

,case_number,year,type,country,activity,age,species_,mes,fatal,sex,especies
0,2018.06.25,2018,Boating,usa,Paddling,57,White shark,Jun,N,F,Tiburón Blanco
1,2018.06.03.a,2018,Unprovoked,brazil,Swimming,18,Tiger shark,Jun,Y,M,Unspecified
2,2018.05.26.b,2018,Unprovoked,usa,Walking,15,"Bull shark, 6'",May,N,M,Unspecified
3,2018.05.24,2018,Provoked,australia,Feeding sharks,32,Grey reef shark,May,N,M,Tiburón Gris
4,2018.05.13.a,2018,Invalid,england,Fishing,21,Invalid incident,May,N,M,Unspecified


2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:

    - Edad en formato string
    - Edades separadas por &, or, to, >
    - Edades con ?

- Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
    - Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
    - Puede que os salga un error similar a este:
    
            TypeError: expected string or bytes-like object
    - Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

            df['nombre_columna'] = df['nombre_columna'].astype(str)
    

- Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.

- Por último cambiad el tipo de la columna de string a integer.

In [64]:
patron_numeros = "\d"

In [65]:
def años_limpios (col):
    try:
        if len(re.findall(patron_numeros, col)) > 0:
            return col
    except:
        return np.nan

In [66]:
ataques["edad"] = ataques["age"].apply(años_limpios)

In [67]:
ataques["edad"].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '60s',
       '51', '61', '59', '42', '6', '27', '23', '29', '39', '24', '12',
       '26', '71', '43', '9', '44', '14', '62', '52', '38', '68', '16',
       '47', '63', '70', '41', '40s', '53', '20s', '7', '66', '45', '74',
       '64', '8', '56', '49', '18 or 20', None, '30s', '77', '8 or 10',
       '84', '30 or 36', '6½', '5', ' 30', ' 28', "60's", '67', '>50',
       '? & 19', '21, 34,24 & 35', '30 & 32', '13 or 18', '7 or 8',
       '9 or 10', nan, '13 or 14'], dtype=object)

In [ ]:
ataques["age"] = ataques["age"].astype("int64")

3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.